* Integrante:
* Morales López Erik Brandon  (20163041)

In [1]:
#Importing the main labries 
import pandas as pd
import numpy as np
import pyreadr
from sklearn import preprocessing
import patsy

from numpy import loadtxt

import hdmpy
import numpy as np
import random
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import itertools
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_categorical_dtype
from itertools import compress
from sklearn.feature_selection import SelectFromModel

# A case of study: Testing the Convergence Hypothesis

# Introduction

We provide an additional empirical example of partialling-out with Lasso to estimate the regression coefficient $\beta_1$ in the high-dimensional linear regression model:
  $$
  Y = \beta_1 D +  \beta_2'W + \epsilon.
  $$
  
Specifically, we are interested in how the rates  at which economies of different countries grow ($Y$) are related to the initial wealth levels in each country ($D$) controlling for country's institutional, educational, and other similar characteristics ($W$).

The outcome $Y$ is the realized annual growth rate of a country's wealth  (Gross Domestic Product per capita). The target regressor ($D$) is the initial level of the country's wealth. The target parameter $\beta_1$ is the speed of convergence, which measures the speed at which poor countries catch up with rich countries. The controls ($W$) include measures of education levels, quality of institutions, trade openness, and political stability in the country.

# Loading the data

In [3]:

data=pyreadr.read_r("Downloads/GrowthData.RData")

# Extracting the data frame from rdata_read
data1 = data[ 'GrowthData' ]
list(data1)

['Outcome',
 'intercept',
 'gdpsh465',
 'bmp1l',
 'freeop',
 'freetar',
 'h65',
 'hm65',
 'hf65',
 'p65',
 'pm65',
 'pf65',
 's65',
 'sm65',
 'sf65',
 'fert65',
 'mort65',
 'lifee065',
 'gpop1',
 'fert1',
 'mort1',
 'invsh41',
 'geetot1',
 'geerec1',
 'gde1',
 'govwb1',
 'govsh41',
 'gvxdxe41',
 'high65',
 'highm65',
 'highf65',
 'highc65',
 'highcm65',
 'highcf65',
 'human65',
 'humanm65',
 'humanf65',
 'hyr65',
 'hyrm65',
 'hyrf65',
 'no65',
 'nom65',
 'nof65',
 'pinstab1',
 'pop65',
 'worker65',
 'pop1565',
 'pop6565',
 'sec65',
 'secm65',
 'secf65',
 'secc65',
 'seccm65',
 'seccf65',
 'syr65',
 'syrm65',
 'syrf65',
 'teapri65',
 'teasec65',
 'ex1',
 'im1',
 'xr65',
 'tot1']

In [4]:
data1.shape

(90, 63)

In [5]:
data1.head(4)

,Outcome,intercept,gdpsh465,bmp1l,freeop,freetar,h65,hm65,hf65,p65,...,seccf65,syr65,syrm65,syrf65,teapri65,teasec65,ex1,im1,xr65,tot1
0,-0.024336,1,6.591674,0.2837,0.153491,0.043888,0.007,0.013,0.001,0.29,...,0.04,0.033,0.057,0.010,47.6,17.3,0.0729,0.0667,0.348,-0.014727
1,0.100473,1,6.829794,0.6141,0.313509,0.061827,0.019,0.032,0.007,0.91,...,0.64,0.173,0.274,0.067,57.1,18.0,0.0940,0.1438,0.525,0.005750
2,0.067051,1,8.895082,0.0000,0.204244,0.009186,0.260,0.325,0.201,1.00,...,18.14,2.573,2.478,2.667,26.5,20.7,0.1741,0.1750,1.082,-0.010040
3,0.064089,1,7.565275,0.1997,0.248714,0.036270,0.061,0.070,0.051,1.00,...,2.63,0.438,0.453,0.424,27.8,22.7,0.1265,0.1496,6.625,-0.002195


# Preprocessing
* To account for heterogeneity across counties and time trends in all variables, we remove from them county-specific and time-specific effects in the following preprocessing

In [6]:
import re

In [7]:
y=data1["Outcome"]
d=data1["gdpsh465"]
z=data1.drop(["Outcome","gdpsh465","intercept"], axis=1)

In [8]:
print(y.shape)
print(d.shape)
print(z.shape)

(90,)
(90,)
(90, 60)


Now, we can construct the treatment variable, the outcome variable and the matrix $Z$ that includes the control variables.

In [9]:
erik = pd.concat([y, d, z], axis=1)
erik.head(4)

,Outcome,gdpsh465,bmp1l,freeop,freetar,h65,hm65,hf65,p65,pm65,...,seccf65,syr65,syrm65,syrf65,teapri65,teasec65,ex1,im1,xr65,tot1
0,-0.024336,6.591674,0.2837,0.153491,0.043888,0.007,0.013,0.001,0.29,0.37,...,0.04,0.033,0.057,0.010,47.6,17.3,0.0729,0.0667,0.348,-0.014727
1,0.100473,6.829794,0.6141,0.313509,0.061827,0.019,0.032,0.007,0.91,1.00,...,0.64,0.173,0.274,0.067,57.1,18.0,0.0940,0.1438,0.525,0.005750
2,0.067051,8.895082,0.0000,0.204244,0.009186,0.260,0.325,0.201,1.00,1.00,...,18.14,2.573,2.478,2.667,26.5,20.7,0.1741,0.1750,1.082,-0.010040
3,0.064089,7.565275,0.1997,0.248714,0.036270,0.061,0.070,0.051,1.00,1.00,...,2.63,0.438,0.453,0.424,27.8,22.7,0.1265,0.1496,6.625,-0.002195


# 1. OLS without including the country characteristics

In [10]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

In [11]:
# OLS clustering at the County level
model = "Outcome ~ gdpsh465"
print(model)
print("formula del OLS:", model)
baseline_ols = smf.ols(model , data=erik).fit().get_robustcov_results()
baseline_ols_table = baseline_ols.summary2().tables[1]
print( baseline_ols_table.iloc[ 1 , 4:] )
baseline_ols_table.iloc[1, :]

Outcome ~ gdpsh465
formula del OLS: Outcome ~ gdpsh465
[0.025   -0.009255
0.975]    0.011888
Name: gdpsh465, dtype: float64


Coef.       0.001317
Std.Err.    0.005320
t           0.247518
P>|t|       0.805084
[0.025     -0.009255
0.975]      0.011888
Name: gdpsh465, dtype: float64

# 2. OLS without including the country characteristics

In [12]:
control_formula = "y ~ d + z"
control_formula

'y ~ d + z'

In [13]:
control_ols = smf.ols( control_formula , data=erik).fit().get_robustcov_results()
control_ols_table = control_ols.summary2().tables[1]
print( control_ols_table.iloc[ 1 , 4:] )
control_ols_table.iloc[1, :]

[0.025   -0.075790
0.975]    0.057034
Name: d, dtype: float64


C:\Users\Erik\anaconda3\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 61, but rank is 60
  warnings.warn('covariance of constraints does not have full '


Coef.      -0.009378
Std.Err.    0.032421
t          -0.289256
P>|t|       0.774517
[0.025     -0.075790
0.975]      0.057034
Name: d, dtype: float64

# DML algorithm

Here we perform inference of the predictive coefficient $\beta$ in our partially linear statistical model, 

$$
Y = D\beta + g(Z) + \epsilon, \quad E (\epsilon | D, Z) = 0,
$$

using the **double machine learning** approach. 

For $\tilde Y = Y- E(Y|Z)$ and $\tilde D= D- E(D|Z)$, we can write
$$
\tilde Y = \alpha \tilde D + \epsilon, \quad E (\epsilon |\tilde D) =0.
$$

Using cross-fitting, we employ modern regression methods
to build estimators $\hat \ell(Z)$ and $\hat m(Z)$ of $\ell(Z):=E(Y|Z)$ and $m(Z):=E(D|Z)$ to obtain the estimates of the residualized quantities:

$$
\tilde Y_i = Y_i  - \hat \ell (Z_i),   \quad \tilde D_i = D_i - \hat m(Z_i), \quad \text{ for each } i = 1,\dots,n.
$$

Finally, using ordinary least squares of $\tilde Y_i$ on $\tilde D_i$, we obtain the 
estimate of $\beta$.

The following algorithm comsumes $Y, D, Z$, and a machine learning method for learning the residuals $\tilde Y$ and $\tilde D$, where the residuals are obtained by cross-validation (cross-fitting). Then, it prints the estimated coefficient $\beta$ and the corresponding standard error from the final OLS regression.

Warning: This DML´s part is too important, because the cluster argument is not necessary for moddeling the algothrim, so I am going to **drop** it !!!

In [18]:
def DML2_for_PLM(z, d, y, dreg, yreg, nfold):
    
    # Num ob observations
    nobs = z.shape[0]
    
    # Define folds indices 
    list_1 = [*range(0, nfold, 1)]*nobs
    sample = np.random.choice(nobs,nobs, replace=False).tolist()
    foldid = [list_1[index] for index in sample]

    # Create split function(similar to R)
    def split(x, f):
        count = max(f) + 1
        return tuple( list(itertools.compress(x, (el == i for el in f))) for i in range(count) ) 

    # Split observation indices into folds 
    list_2 = [*range(0, nobs, 1)]
    I = split(list_2, foldid)
    
    # Create array to save errors 
    dtil = np.zeros( len(z) ).reshape( len(z) , 1 )
    ytil = np.zeros( len(z) ).reshape( len(z) , 1 )
    
    # loop to save results
    for b in range(0,len(I)):
    
        # Split data - index to keep are in mask as booleans
        include_idx = set(I[b])  #Here should go I[b] Set is more efficient, but doesn't reorder your elements if that is desireable
        mask = np.array([(i in include_idx) for i in range(len(z))])

        # Lasso regression, excluding folds selected 
        dfit = dreg(z[~mask,], d[~mask,])
        yfit = yreg(z[~mask,], y[~mask,])

        # predict estimates using the 
        dhat = dfit.predict( z[mask,] )
        yhat = yfit.predict( z[mask,] )

        # save errors  
        dtil[mask] =  d[mask,] - dhat.reshape( len(I[b]) , 1 )
        ytil[mask] = y[mask,] - yhat.reshape( len(I[b]) , 1 )
        print(b, " ")
    
    # Create dataframe 
    data_2 = pd.DataFrame(np.concatenate( ( ytil, dtil ), axis = 1), columns = ['ytil','dtil'])
   
    # OLS clustering at the County level
    model = "ytil ~ dtil"
    baseline_ols = smf.ols(model , data = data_2 ).fit().get_robustcov_results()
    coef_est = baseline_ols.summary2().tables[1]['Coef.']['dtil']
    se = baseline_ols.summary2().tables[1]['Std.Err.']['dtil']
    
    Final_result = { 'coef_est' : coef_est , 'se' : se , 'dtil' : dtil , 'ytil' : ytil }

    print("Coefficient is {}, SE is equal to {}".format(coef_est, se))
    
    return Final_result

In [19]:
# Create main variables

y=data1["Outcome"]
d=data1["gdpsh465"]
z=data1.drop(["Outcome","gdpsh465","intercept"], axis=1)

In [20]:
# as matrix
y = y.to_numpy().reshape( len(y) , 1 )
d = d.to_numpy().reshape( len(y) , 1 )
z = z.to_numpy()

In [21]:
z.shape

(90, 60)

In [22]:
d.shape

(90, 1)

In the following, we apply the DML approach with the differnt versions of lasso.

# 3. Lasso Using scikit-learn : DML using Lasso to predict y an d.

In [24]:

def dreg(z,d):
    alpha=0.00000001
    result = linear_model.Lasso(alpha = alpha).fit(z, d)
    return result

def yreg(z,y):
    alpha=0.00000001
    result = linear_model.Lasso(alpha = alpha).fit(z, y)
    return result

DML2_lasso = DML2_for_PLM(z, d, y, dreg, yreg, 10)

0  
1  
2  
3  
4  
5  
6  
7  
8  
9  
Coefficient is 0.022253809969034144, SE is equal to 0.012372169913013162


C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.846871087619321, tolerance: 0.006657853775832402
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.012672435153956238, tolerance: 2.1126353101377927e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7435725876314743, tolerance: 0.006369264804598845
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: Conver

# 4. Post - Lasso Using scikit-learn: DML using Post-Lasso to predict y an d

In [25]:
class Lasso_post:
    
    def __init__(self, alpha ):
        self.alpha = alpha

        
    def fit( self, X, Y ):
        self.X = X
        self.Y = Y
        lasso = linear_model.Lasso( alpha = self.alpha ).fit( X , Y )
        model = SelectFromModel( lasso , prefit = True )
        X_new = model.transform( X )
        # Gettin indices from columns which has variance for regression
        index_X = model.get_support()
        
        self.index = index_X
        new_x = X[ : ,  index_X ]
        
        lasso2 = linear_model.Lasso( alpha = self.alpha ).fit( new_x , Y )
        self.model = lasso2
        
        return self
    
    def predict( self , X ):
        
        dropped_X = X[ : , self.index ]
        
        predictions = self.model.predict( dropped_X )
        
        return predictions

In [26]:
z.shape

(90, 60)

In [27]:
d.shape

(90, 1)

In [32]:
def dreg( z, d ):
    alpha = 0.00000001
    result = Lasso_post( alpha = alpha ).fit( z , d )
    return result

def yreg( z , y ):
    alpha = 0.00000001
    result = Lasso_post( alpha = alpha ).fit( z , y )
    return result

DML2_lasso_post = DML2_for_PLM(z, d, y, dreg, yreg, 10)

C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7042587005741707, tolerance: 0.006712857010913807
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7092973584731672, tolerance: 0.006712857010913807
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.013249918089594071, tolerance: 1.981152704868287e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: Conver

0  
1  
2  
3  
4  
5  
6  
7  
8  
9  
Coefficient is 0.033532902336266705, SE is equal to 0.0181377302391674


C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.010122020292847526, tolerance: 2.1248934702264444e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.5479149486228192, tolerance: 0.006256317524255638
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.543584275743984, tolerance: 0.006256317524255638
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: Conver

# 5. DML using Elastic Net to predict y an d

In [33]:
class standard_skl_model:
    
    def __init__(self, model ):
        self.model = model
       
    def fit( self, X, Y ):
        
        # Standarization of X and Y
        self.scaler_X = StandardScaler()
        self.scaler_X.fit( X )
        std_X = self.scaler_X.transform( X )
                
        self.model.fit( std_X , Y )
                
        return self
    
    def predict( self , X ):
        
        self.scaler_X = StandardScaler()
        self.scaler_X.fit( X )
        std_X = self.scaler_X.transform( X )
        
        prediction = self.model.predict( std_X )
        
        return prediction

In [34]:
# DML with cross-validated Elastic Net:
def dreg(z,d):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.5, max_iter = 100000 ) ).fit( z, d )
    return result

def yreg(z,y):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.5, max_iter = 100000 ) ).fit( z, y )
    return result

DML2_elnet = DML2_for_PLM(z, d, y, dreg, yreg, 2)

C:\Users\Erik\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0  


C:\Users\Erik\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


1  
Coefficient is -0.04173911736135387, SE is equal to 0.013956988482804716


# 6. DML using Ridge to predict y an d

In [35]:
#DML with cross-validated Ridge:
def dreg(z,d):
    result = standard_skl_model( ElasticNetCV( cv = 10 ,  random_state = 0 , l1_ratio = 0.0001 ) ).fit( z, d )
    return result

def yreg(z,y):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.0001 ) ).fit( z, y )
    return result

DML2_ridge = DML2_for_PLM(z, d, y, dreg, yreg, 2)

C:\Users\Erik\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0  


C:\Users\Erik\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


1  
Coefficient is -0.03706592802939041, SE is equal to 0.012535710772729056


# 7. Optional :DML with OLS

In [36]:
#DML with OLS:
def dreg(z,d):
    result = LinearRegression().fit( z, d )
    return result

def yreg(z,y):
    result = LinearRegression().fit( z, y )
    return result

DML2_ols = DML2_for_PLM(z, d, y, dreg, yreg, 2)

0  
1  
Coefficient is -0.056295192024286525, SE is equal to 0.016906185689406196


# 8. Random Forest: DML using Random Forest to predict y an d

In [37]:
# Importing the main libraries
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [38]:
#DML with Random Forest:
def dreg(z,d):
    result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 65 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, d )
    return result

def yreg(z,y):
    result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 65 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, y )
    return result

DML2_RF = DML2_for_PLM(z, d, y, dreg, yreg, 2)   # set to 2 due to computation time

<ipython-input-38-f459882d479f>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 65 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, d )


ValueError: max_features must be in (0, n_features]

I having some issue running the random forest

In [39]:
class standard_skl_model:
    
    def __init__(self, model ):
        self.model = model
       
    def fit( self, X, Y ):
        
        # Standarization of X and Y
        self.scaler_X = StandardScaler()
        self.scaler_X.fit( X )
        std_X = self.scaler_X.transform( X )
                
        self.model.fit( std_X , Y )
                
        return self
    
    def predict( self , X ):
        
        self.scaler_X = StandardScaler()
        self.scaler_X.fit( X )
        std_X = self.scaler_X.transform( X )
        
        prediction = self.model.predict( std_X )
        
        return prediction

In [40]:
#DML with cross-validated Lasso:
def dreg(z,d):
    result = standard_skl_model( LassoCV(cv = 10 , random_state = 0 ) ).fit( z, d )
    return result

def yreg(z,y):
    result = standard_skl_model( LassoCV(cv = 10 , random_state = 0  ) ).fit( z, y )
    return result

DML2_lasso_cv = DML2_for_PLM(z, d, y, dreg, yreg, 2)

C:\Users\Erik\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0055245620700798415, tolerance: 0.0031524738673747873
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.003229897842483176, tolerance: 0.0028726822286616127
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not conver

C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.00010698527669520145, tolerance: 1.0529025840562e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0001188635507494895, tolerance: 1.0529025840562e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.00015173340066748338, tolerance: 1.0529025840562e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: C

C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.355590491924313e-05, tolerance: 9.801451023672357e-06
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.686376255814894e-05, tolerance: 9.801451023672357e-06
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.7257672137855912e-05, tolerance: 9.801451023672357e-06
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:5

C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.00010134715296808341, tolerance: 1.089066738000148e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.118211455917195e-05, tolerance: 1.089066738000148e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.205856502380141e-05, tolerance: 1.089066738000148e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:5

C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.711671108440649e-05, tolerance: 9.207607035310561e-06
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.0786256111388676e-05, tolerance: 9.207607035310561e-06
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.1986460155215353e-05, tolerance: 9.207607035310561e-06
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:

...

C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.626117625785301e-05, tolerance: 9.967695524722944e-06
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.585957828097791e-05, tolerance: 9.967695524722944e-06
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.27535000181402e-05, tolerance: 9.967695524722944e-06
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530

C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.975988800875927e-05, tolerance: 1.2219860372108855e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.836321602523508e-05, tolerance: 1.2219860372108855e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.753361222541603e-05, tolerance: 1.2219860372108855e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py

C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.00020264846851199888, tolerance: 1.2219860372108855e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.00018643149211744784, tolerance: 1.2219860372108855e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.00017371648297094294, tolerance: 1.2219860372108855e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent

1  
Coefficient is -0.02278336360518967, SE is equal to 0.010008677167618258


C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.8978504204535986e-05, tolerance: 1.2232267848600146e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.9893840365979056e-05, tolerance: 1.2232267848600146e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.4676907686874535e-05, tolerance: 1.2232267848600146e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent

In [41]:
mods = [DML2_ols, DML2_lasso, DML2_lasso_post , DML2_lasso_cv, DML2_ridge, DML2_elnet]
mods_name = ["DML2_ols", "DML2_lasso", "DML2_lasso_post" , "DML2_lasso_cv", 'DML2_ridge', 'DML2_elnet']

def mdl( model , model_name ):
    
    RMSEY = np.sqrt( np.mean( model[ 'ytil' ] ) ** 2 ) # I have some doubts about these equations...we have to recheck
    RMSED = np.sqrt( np.mean( model[ 'dtil' ] ) ** 2 ) # I have some doubts about these equations...we have to recheck
    
    result = pd.DataFrame( { model_name : [ RMSEY , RMSED ]} , index = [ 'RMSEY' , 'RMSED' ])
    return result

RES = [ mdl( model , name ) for model, name in zip( mods , mods_name ) ]
    

pr_Res = pd.concat( RES, axis = 1)

pr_Res

,DML2_ols,DML2_lasso,DML2_lasso_post,DML2_lasso_cv,DML2_ridge,DML2_elnet
RMSEY,0.028997,0.002658,0.001319,3.777842e-18,1.464878e-18,6.630499e-18
RMSED,0.112237,0.035543,0.017498,1.944124e-15,5.921189e-16,9.868649e-18


In [42]:
class standard_skl_model:
    
    def __init__(self, model ):
        self.model = model
       
    def fit( self, X, Y ):
        
        # Standarization of X and Y
        self.scaler_X = StandardScaler()
        self.scaler_X.fit( X )
        std_X = self.scaler_X.transform( X )
                
        self.model.fit( std_X , Y )
                
        return self
    
    def predict( self , X ):
        
        self.scaler_X = StandardScaler()
        self.scaler_X.fit( X )
        std_X = self.scaler_X.transform( X )
        
        prediction = self.model.predict( std_X )
        
        return prediction

In [43]:
#DML with the BEST:
def dreg(z,d):
    result = standard_skl_model( LassoCV(cv = 10 , random_state = 0 , alphas = [0]) ).fit( z, d )
    return result


def yreg(z,y):
    result = standard_skl_model( ElasticNetCV( cv = 10 ,  random_state = 0 , l1_ratio = 0.0001 ) ).fit( z, y )
    return result

DML2_best = DML2_for_PLM(z, d, y , dreg, yreg, 10)

C:\Users\Erik\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.45525610963471763, tolerance: 0.005994993780042701
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_f

0  


C:\Users\Erik\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.3219261177846704, tolerance: 0.006062958106357144
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fa

1  


C:\Users\Erik\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.5392156274730979, tolerance: 0.005959611446549626
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fa

...

7  


C:\Users\Erik\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.45690109010514846, tolerance: 0.005641492364959308
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_f

8  


C:\Users\Erik\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.5409982169260132, tolerance: 0.005990296046159705
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Erik\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fa

9  
Coefficient is -0.006918334004719106, SE is equal to 0.005862449228460352


# The results

In [44]:
table = np.zeros( ( 8 , 2 ))
table[ 0 , 0] = baseline_ols_table.iloc[ 1 , 0 ]
table[ 1 , 0] = control_ols_table.iloc[ 1 , 0 ]
table[ 2 , 0] = DML2_lasso['coef_est']
table[ 3 , 0] = DML2_lasso_post['coef_est']
table[ 4 , 0] = DML2_lasso_cv['coef_est']
table[ 5 , 0] = DML2_ridge['coef_est']
table[ 6 , 0] = DML2_elnet['coef_est']
table[ 7 , 0] = DML2_best['coef_est']
table[ 0 , 1] = baseline_ols_table.iloc[ 1 , 1 ]
table[ 1 , 1] = control_ols_table.iloc[ 1 , 1 ]
table[ 2 , 1] = DML2_lasso['se']
table[ 3 , 1] = DML2_lasso_post['se']
table[ 4 , 1] = DML2_lasso_cv['se']
table[ 5 , 1] = DML2_ridge['se']
table[ 6 , 1] = DML2_elnet['se']
table[ 7 , 1] = DML2_best['se']

In [45]:
table = pd.DataFrame( table , index = [ "Baseline OLS", "Least Squares with controls", "Lasso", \
             "Post-Lasso", "CV Lasso","CV Elnet", "CV Ridge", "Best" ] , \
            columns = ["Estimate","Standard Error"] )
table.round( 3 )

,Estimate,Standard Error
Baseline OLS,0.001,0.005
Least Squares with controls,-0.009,0.032
Lasso,0.022,0.012
Post-Lasso,0.034,0.018
CV Lasso,-0.023,0.010
CV Elnet,-0.037,0.013
CV Ridge,-0.042,0.014
Best,-0.007,0.006
